# Blibiotecas

In [ ]:
!pip install google-search-results

In [ ]:
import pandas as pd
from serpapi import GoogleSearch
import ast
import re

# Recolhendo dados principais dos estabelecimentos
O primeiro tipo de dados que vamos obter são as infromações básicas dos restaurantes, como: localização, nota média, faixa de preço, categoria, entre outros.
<br>
Algumas etapas são necessaŕias para alcançar nosso objetivo:


1.   Definir quais termos iremos passar como requisição na API para realizar pesquisas no Google Maps
2.   Realizar as requisições e armazenar os resultados
3. Passar os dados obtidos para Pandas DataFrame e aplicar as transformações necessaŕias a fim de facilitar o manuseio deles no decorrer do projeto
4. Armazenar os dados





## Funções

In [ ]:
def buscar_na_api(pesquisas, key):

  '''
  Recebe a lista dos termos a serem pesquisados no Google Maps juntamente com a chave individual que possibilita que a requsição seja feita.
  Retorna lista de listas com json dos resultados obtidos, requisições que falharam e todas as pesquisas realizadas.

  pesquisas = Termos a ser pesquisado
  key = Chave individual da API
  '''
  local_results = []
  local_fail = []
  local_titles = []

  for pesquisa in pesquisas:
    print(pesquisa)
    for pg in range(0, 101, 20):
      params = {
        "engine": "google_maps",
        "q": pesquisa,
        "type": "search",
        "api_key": key,
        "start":pg,
        "ll": '@-23.5489,-46.6388,7z'
      }

      search = GoogleSearch(params)
      results = search.get_dict()
      try:
        local_results = (local_results + results["local_results"])
        local_titles.append(pesquisa)
      except:
        print('fail')
        if results['error'] == 'Your searches for the month are exhausted. You can upgrade plans on SerpApi.com website.':
          print(f'O último erro foi:\n{results}')
          return [local_results, local_fail, local_titles]
        else:
          local_titles.append(pesquisa)
          local_fail.append(pesquisa)
          break


  return [local_results, local_fail, local_titles]

def criando_pesquisas(locais_pesquisas, regioes_sp):
  '''
  Recebe e combina strings para retornar diversas variações de pesquisa
  locais_pesquisas = categorias pesquisadas
  regioes_sp = localização a ser pesquisada
  '''
  lista_pesquisas = []

  for l in locais_pesquisas:
    for r in regioes_sp:
      lista_pesquisas.append(l+r)
  
  return lista_pesquisas

## Definindo Pesquisas

In [ ]:
local_results = []

locais_pesquisas = [
    "Churrascaria",
    "Almoço",
    "Jantar Romântico",
    "Jantar",
    "Dogão",
    "Cachorro quente",
    "Lanche",
    "Sanduiche",
    "Prato Feito",
    "PF",
    "Sorveteria",
    "Balada",
    "Bolos",
    "Rodízio de Massas",
    "Rodízio de Doces",
    "Rodízio de Pizza",
    "Self-Service",
    "Comida de Boteco",
    "Restuarante com Vista",
    "Comida Mineira",
    "Feijoada",
    "Salada",
    "Comida Saudavel",
    "Marmita",
    "Balada Alternativa",
    "Restaurante Diferente",
    "Doceria Gourmet",
    "Petiscos",
    "Brunch",
    "Galeteria",
    "Doce Francês",
    "Doce Árabe",
    "Restaurante de Comida Chinesa",
    "Restaurante Alternativo",
    "Café Alternativo",
    "Rodízio de Drinks",
     "Comida do Norte",
    "Café em São Paulo",
    "Restaurante em São Paulo",
    "Pizzaria em São Paulo",
    "Pizzaria Napolitana em São Paulo",
    "Hamburgueria em São Paulo",
    "Rodízio de Carne em São Paulo",
    "Rodízio de Comida Japones em São Paulo",
    "Rodízio em São Paulo",
    "Bar em São Paulo",
    "Barzinho em São Paulo",
    "Pub em São Paulo",
    "Trattoria em São Paulo",
    "Padaria em São Paulo",
    "Doceria em São Paulo",
    "Boteco em São Paulo",
    "Balada em São Paulo",
    "Restaurante Italiano em São Paulo",
    "Restaurante Japonês em São Paulo",
    "Restaurante Chinês em São Paulo",
    "Restaurante Mediterrâneo em São Paulo",
    "Restaurante Árabe em São Paulo",
    "Restaurante Nordestino em São Paulo",
    "Restaurante Alemão em São Paulo",
    "Restaurante Tailandês",
    "Restaurante de Comida Australiana",
    "Restaurante de Comida Coreana",
    "Restaurante de Comida Indiana",
    "Churrascaria Argentina",
    "Churrascaria Gaúcha"
]

regioes_sp = ["", " em São Paulo Zona Norte", " em São Paulo Zona Sul", " em São Paulo", " Barato", " Caro"]

pesquisas = criando_pesquisas(locais_pesquisas, regioes_sp)

## Realizando Requisições

In [ ]:
key = '29c08d9dcd009ac3c8fe92404122edd4f29225ae6fd19b8163b705629aa1b7f5' # O limite de consultas foi esgotado
local_results = buscar_na_api(pesquisas, key)

## Tratando dados

In [ ]:
# dataset principal

overview = pd.DataFrame(local_results[0])

# dividindo dados de latitude e longitude em colunas distintas
gps_coordinates = pd.concat([pd.DataFrame([coo]) for coo in overview['gps_coordinates']]).reset_index()
overview = pd.concat([overview, gps_coordinates], axis=1)

# definindo colunas que irão compor o dataset
df_overview = overview[['position', 'title', 'place_id', 'data_id', 'data_cid', 'rating',
       'reviews', 'price', 'type', 'address', 'latitude', 'longitude', 'open_state',
       'description', 'thumbnail']]

In [ ]:
# dataset horarios de funcionamento

# dispondo dias e seus horarios de funcionamento em colunas distintas
titles = overview[['title',	'place_id']]
operating_hours = pd.concat([pd.DataFrame([oph]) for oph in overview['operating_hours']]).reset_index()
df_operating_hours = pd.concat([titles, operating_hours], axis=1)

# definindo colunas que irão compor o dataset
df_operating_hours =  df_operating_hours[['title', 'place_id',  'sunday',    'monday', 'tuesday', 'wednesday', 'thursday',  'friday',  'saturday']]

In [ ]:
# dataset de opções de serviço

# dispondo colunas de opção de serviço em colunas distintas
service_options = pd.concat([pd.DataFrame([so]) for so in overview['service_options']]).reset_index()
df_service_options = pd.concat([titles, service_options], axis=1)

# definindo colunas que irão compor o dataset
df_service_options = df_service_options[['title', 'place_id','index','dine_in','takeout','delivery',
           'curbside_pickup', 'no_contact_delivery','drive_through','takeaway','kerbside_pickup',
           'in_store_shopping','in_store_pickup'
           ]]

## Salvando Dados

In [ ]:
df_overview = df_overview.drop_duplicates()
df_overview.to_csv(f'df_overview (1).csv', index=False)

df_operating_hours = df_operating_hours.drop_duplicates()
df_operating_hours.to_csv(f'df_operating_hours (1).csv', index=False)

df_service_options = df_service_options.drop_duplicates()
df_service_options.to_csv(f'df_service_options (1).csv', index=False)

# Recolhendo dados de Review
Os dados de Review trazem infromações mais de talhadas dos estabelecimentos, como comentários, avaliações, horários de movimento, entre outros.<br> Diferente das informações básicas recolhidas anteriormente, nem todos os estabelcimentos terão esse tipo de dado.
<br>
Algumas etapas são necessaŕias para alcançar nosso objetivo:


1.   Definir quais estabelecimentos serão usados na consulta dentre os presentes nos Dados Gerais. 
> *Não é possível de recolher de todos. Isso porque temos uma limitação de infraestrutura referente ao limite de requisições permitidas por usuário na API. <br>
Para ilustrar, caso quisessemos pegar os dados de review dos mais de 8 mil restaurantes presentes nos Dados Gerais, precisariamos de mais de 80 Chaves individuais, o que seria inviável. <br>
Outra alternativa seria utilizar a versão paga da API, porém, levando em conta o tipo e finalidade deste projeto, também não se mostra viável.*


2.   Realizar as requisições e armazenar os resultados
3. Passar os dados obtidos para Pandas DataFrame e aplicar as transformações necessaŕias a fim de facilitar o manuseio deles no decorrer do projeto
4. Armazenar os dados

## Funções

In [ ]:
def get_review_info(params):
  c = 0
  search = GoogleSearch(params)
  results = search.get_dict()
  print(results)
  try:
    place_results = results["place_results"]
    return place_results
  except:
    print('fail')

In [ ]:
def back_to_json(txt, chave1, chave2=None):
  new_txt = txt
  # new_txt = re.sub('"', 'aspas', txt)
  new_txt = re.sub('\'', '"', new_txt)
  # new_txt = re.sub('aspas', '\'', new_txt)
  if chave2 == None:
    return ast.literal_eval(new_txt)[chave1]
  else:
    return ast.literal_eval(new_txt)[chave1][chave2]

## Definindo Restaurantes

In [ ]:
# Devido à limitação de infraestrutura da API apresentar um pequeno limite de requisições por usário, só pudemos recolher informações de review de uma amostra dos nossos dados.
place_id_list = df_overview.sample(600)
place_id_list = list(place_id_list.place_id.values)

## Realizando Requisições

In [ ]:
place_results_list = []

for p in place_id_list:
    print(p)
    params = {
      "engine": "google_maps",
      "type": "place",
      "place_id": p,
      "api_key": key
    }
    place_results_list.append(get_review_info(params))

df_review_total = pd.DataFrame(place_results_list)

## Tratando Dados

In [ ]:
overview_avaliacoes_list = []
most_relevant_reviews_list = []
tambem_procuram_list = []
horarios_movimento_list = [] 

for index, row in  df_review_total.iterrows():

  # tabela do overview das avaliacoes
  try:
    overview_avaliacoes = pd.DataFrame({
        'title': row['title'], 
        'place_id': row['place_id'], 
        'user_reviews(summary)': row['user_reviews']['summary'], 
        'time_spent': row['popular_times']['live_hash']['time_spent']
        })
    overview_avaliacoes_list.append(overview_avaliacoes)
  except:
    pass


  # tabela de reviews mais relevantes
  try:
    most_relevant_reviews = pd.DataFrame(row['user_reviews']['most_relevant'])
    most_relevant_reviews['title'] = row['title']
    most_relevant_reviews['place_id'] = row['place_id']
    most_relevant_reviews_list.append(most_relevant_reviews)
  except:
    pass

  # tabela de correlação entre os restaurantes
  try:
    tambem_procuram = pd.DataFrame(row['people_also_search_for'][0]['local_results'])
    tambem_procuram['estabelecimento_referencia'] = row['title']
    gps_coordinates = pd.concat([pd.DataFrame([coo]) for coo in tambem_procuram['gps_coordinates']]).reset_index()
    tambem_procuram = pd.concat([tambem_procuram, gps_coordinates], axis=1)


    tambem_procuram_list.append(tambem_procuram)
  except:
    pass


  # tabela de grafico de horáros
  try:
    horarios_movimento = pd.DataFrame(row['popular_times']['graph_results'])
    horarios_movimento['title'] = row['title']
    horarios_movimento_list.append(horarios_movimento)
  except:
    pass

## Salvando Dados

In [ ]:
overview_avaliacoes = pd.concat(overview_avaliacoes_list)
overview_avaliacoes = overview_avaliacoes.astype(str).drop_duplicates()
overview_avaliacoes.to_csv('overview_avaliacoes.csv', index=False)

most_relevant_reviews = pd.concat(most_relevant_reviews_list)
most_relevant_reviews = most_relevant_reviews[['title', 'place_id', 'username', 'rating', 'description', 'images', 'date']]
most_relevant_reviews = most_relevant_reviews.astype(str).drop_duplicates()
most_relevant_reviews.to_csv('most_relevant_reviews.csv', index=False)

tambem_procuram = pd.concat(tambem_procuram_list)
tambem_procuram = tambem_procuram[['estabelecimento_referencia', 'position', 'title', 'rating',
      'reviews', 'type', 'thumbnail', 'latitude', 'longitude',]]
tambem_procuram = tambem_procuram.astype(str).drop_duplicates()
tambem_procuram.to_csv('tambem_procuram.csv', index=False)

horarios_movimento = pd.concat(horarios_movimento_list)
horarios_movimento = horarios_movimento.astype(str).drop_duplicates()
horarios_movimento.to_csv('horarios_movimento.csv', index=False)